In [5]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
#import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math
import dill as pickle

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.core import VoxelModelCache
from mcmodels.connectivity.creation import get_connectivity_matrices3
from mcmodels.core.connectivity_data import get_connectivity_data
#from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures, get_leaves_ontologicalorder, get_indices_2ormore, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3
from mcmodels.regressors import NadarayaWatson
from mcmodels.core.plotting import plot_loss_surface,plot_loss_scatter
from mcmodels.models.expectedloss.crossvalidation import get_loss_surface_cv_spline
from mcmodels.models.expectedloss.crossvalidation import get_embedding_cv
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss, Crossval
from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists,CrossvalNW

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/default_structures', default_structures)
#np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/ontological_order_v3', ontological_order)
#np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/default_structure_ids', default_structure_ids)
#major_structures = input_data.get('structures')
#major_structure_ids = [get_structure_id(cache, s) for s in major_structures]

In [7]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
input_data = ju.read(INPUT_JSON)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
cache = VoxelModelCache(manifest_file=manifest_file)
st = cache.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}
major_structures = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/major_structures.npy')
major_structure_ids = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/major_structure_ids.npy')
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
ontological_order_leaves = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/ontological_order_leaves_v3.npy')

annotation/ccf_2017


In [8]:
connectivity_data = get_connectivity_data(cache, major_structure_ids, experiments_exclude, remove_injection = False, structure_set_id=167587189)
connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()
connectivity_data.get_data_matrices(major_structure_ids)

connectivity_data.ai_map = ai_map
connectivity_data.get_crelines(data_info)
connectivity_data.get_summarystructures(data_info)
connectivity_data.summary_structures = {sid: connectivity_data.structure_datas[sid].summary_structures for sid in major_structure_ids}#get_indices_2ormore(connectivity_data.leafs)
connectivity_data.leafs = leafs

sid0 = list(connectivity_data.structure_datas.keys())[0]
#Identify keys denoting which voxels correspond to which structure in the ipsi and contra targets.
targ_ord = ontological_order_leaves
source_ord = ontological_order_leaves
contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=targ_ord, hemisphere_id=1)
#ontological_order = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/ontological_order_v3.npy')
ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=targ_ord, hemisphere_id=2)
connectivity_data.get_regionalized_normalized_data(source_ord, ipsi_targetkey, contra_targetkey)

frac_learn = np.ones(12)
connectivity_data.sum2_index_matrices = get_indices_2ormore(connectivity_data.summary_structures)
connectivity_data.get_cresum_combos()
connectivity_data.cresum2_index_matrices = get_indices_2ormore(connectivity_data.cresum_combos)
connectivity_data.cresum2_evalindices = get_eval_indices(connectivity_data.cresum2_index_matrices)

for m in range(12):
    sid = major_structure_ids[m]
    connectivity_data.structure_datas[sid].loss_surface_cv_str = get_loss_surface_cv_spline(connectivity_data.structure_datas[sid].reg_proj_norm,
                                                                                                 connectivity_data.structure_datas[sid].centroids,
                                                                                                 connectivity_data.creline[sid],
                                                                                                 connectivity_data.summary_structures[sid],
                                                                                                 frac_learn[m])
    connectivity_data.structure_datas[sid].smoothed_losses_str = get_embedding_cv(surface = connectivity_data.structure_datas[sid].loss_surface_cv_str,
                                                                              dists = pairwise_distances(connectivity_data.structure_datas[sid].centroids)**2,
                                                                              cre_distances_cv = connectivity_data.structure_datas[sid].loss_surface_cv_str.cre_distances_cv)
    
gammas = np.asarray([0.0001,0.001,0.01,0.1,1,10,100,1000,10000])
reg_proj_norm = {sid: connectivity_data.structure_datas[sid].reg_proj_norm for sid in major_structure_ids}
sum2_index_matrices = {sid: connectivity_data.sum2_index_matrices[sid] for sid in major_structure_ids} 
cresum2_evalindices = {sid: connectivity_data.cresum2_evalindices[sid] for sid in major_structure_ids} 
distances = {sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
crelines = {sid: connectivity_data.creline[sid] for sid in major_structure_ids}



512
703
1089
1097
315
313
354
698
771
803
477
549


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/core/connectivity_data.py:225: RuntimeWarning: divide by zero encountered in true_divide
  np.sum(reg_inj_vcount_norm, axis=1), 1)
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/core/connectivity_data.py:225: RuntimeWarning: invalid value encountered in true_divide
  np.sum(reg_inj_vcount_norm, axis=1), 1)
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/core/connectivity_data.py:229: RuntimeWarning: divide by zero encountered in true_divide
  np.sum(reg_ipsi_inj, axis=1), 1)
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/core/connectivity_data.py:229: RuntimeWarning: invalid value encountered in true_divide
  np.sum(reg_ipsi_inj, axis=1), 1)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Use

In [26]:
cresum2_evalindices[512]

array([ 1,  4,  7,  9, 11, 17, 21, 27, 29, 31])

In [9]:
connectivity_data.get_creleaf_combos()

In [10]:
twostage_leaf_creleaf2 = CrossvalNW(reg_proj_norm, distances,sum2_index_matrices, cresum2_evalindices,gammas)
twostage_leaf_creleaf2.gammas = gammas #ds
twostage_leaf_creleaf2.predictions = twostage_leaf_creleaf2.get_predictions()
twostage_leaf_creleaf2.get_results_loocv()
twostage_leaf_creleaf2.get_results_weightedloocv(connectivity_data.summary_structures,crelines,ia_map)


In [17]:
twostage_leaf_creleaf2.meanloss

array([0.41710553, 0.75822031, 0.15659895, 0.32778434, 0.21333922,
       0.26689283, 0.19494844, 0.1217142 , 0.30690343, 0.19538653,
       0.09545903, 0.34220301])

In [24]:
np.nanmean(distances[512])

0.29421137505808076

In [11]:
connectivity_data.leaf2_index_matrices = get_indices_2ormore(connectivity_data.leafs)

In [12]:
connectivity_data.leaf2_index_matrices[512].sum(axis = 0)

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [13]:
connectivity_data.creleaf2_index_matrices = get_indices_2ormore(connectivity_data.creleaf_combos)

In [14]:
connectivity_data.structure_datas[512].reg_proj_norm.shape, connectivity_data.creleaf2_index_matrices[512].shape

((36, 1123), (34, 36))

In [15]:
connectivity_data.creleaf2_index_matrices[512].sum(axis = 1)

array([0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
connectivity_data.cresum2_index_matrices[512].sum(axis = 1)

array([0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
cresum2_index_matrices = {sid: connectivity_data.cresum2_index_matrices[sid] for sid in major_structure_ids} 
distances = pds#{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
nw_creleaf_creleaf2 = CrossvalNW(reg_proj_norm, distances,cresum2_index_matrices, cresum2_evalindices,gammas)
nw_creleaf_creleaf2.gammas = gammas# ds
nw_creleaf_creleaf2.predictions = nw_creleaf_creleaf2.get_predictions()
nw_creleaf_creleaf2.get_results_loocv()
nw_creleaf_creleaf2.get_results_weightedloocv(connectivity_data.summary_structures,crelines,ia_map)

In [ ]:
surfaces = {}
for m in range(len(major_structure_ids)):
    sid = major_structure_ids[m]
    surfaces[sid ] = connectivity_data.structure_datas[sid].loss_surface_cv_str
    surfaces[sid ].gamma = twostage_leaf_creleaf2.bestgamma[m]
    
with open('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/trainedmodels/ELsummary_surface_0407_leafleaf.pickle', 'wb') as handle:
    pickle.dump(surfaces, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
nw_creleaf_creleaf2.meanloss_weighted

In [ ]:
twostage_leaf_creleaf2.meanloss_weighted

In [ ]:
np.where(crelines[315] == 'A930038C07Rik-Tg1-Cre')[0]